In [1]:
import esoreader as eso
import pandas as pd
import matplotlib.pyplot as plt
import plotly.io as pio

import os

from testbench_new import main

pd.options.plotting.backend = "plotly"
pio.renderers.default = 'iframe'

### Configuration
uses always the default configuration which is overwritten by "use_set". E.g. "use_set": "deploy" uses the "deploy" set to overwrite the "default" set

In [2]:
config = {
  "mode": "deploy",
  "default": {
    "control_type": "HEATING",
    "work_hours_start_time": "8:00",
    "work_hours_end_time": "18:00",
    "heating_high_setpoint": 23.0,
    "heating_low_setpoint": 20.0,
    "working_on_saturdays": False,
    "smoothing_function_name": "MEAN",
    "smoothing_function_sigma": 1,
    "smoothing_function_mode": "nearest",
    "smoothing_function_return_index": "MIDDLE",
    "window_size_in_minutes": 31,
    "window_offset": "PRESENT",
    "tz": "Europe/Vienna",
    "country": "AT",
    "timesteps_per_hour": 60,
    "building_file": "5zone_unitary_heat_only",
    "weather_file": "const",
    "visualize": ["Outdoor Temperature","Indoor Temperature","Boiler Outlet Temperature", "Setpoint"]
  },
  "analyse": {
    "trials": 1,
    "hours_per_trial": 120,
    "reset_after_trial": False,
    "optimize": False
  },
  "evaluate": {
    "trials": 300,
    "hours_per_trial": 120,
    "reset_after_trial": True,
    "optimize": True
  },
  "deploy": {
    "trials": 14,
    "hours_per_trial": 12,
    "reset_after_trial": False,
    "optimize": True
  }
}

In [3]:
def create_dataframe_from_eso(eso_file):
    """
    Creates a dataframe from an eso file
    """

    df_temp = pd.DataFrame()
    df_control = pd.DataFrame()
    
    df_outdoor_temp = eso_file.to_frame("Site Outdoor Air Drybulb Temperature")
    df_outdoor_temp.columns = ["Outdoor Temperature"]
    
    df_indoor_temp = eso_file.to_frame("Zone Air Temperature")
    df_indoor_temp.columns = ["Indoor Temperature"]

    df_heat_coil = eso_file.to_frame("Heating Coil Heating Energy")
    df_heat_coil.columns = ["Heating Coil Energy"]
    df_heat_coil["Heating Coil Energy"] = df_heat_coil["Heating Coil Energy"]/1000000

    df_boiler_out = eso_file.to_frame("Boiler Outlet Temperature")
    df_boiler_out.columns = ["Boiler Outlet Temperature"]

    df_solar_dir = eso_file.to_frame("Site Direct Solar Radiation Rate per Area")
    df_solar_dir.columns = ["Direct Solar"]
    df_solar_dir["Direct Solar"] = df_solar_dir["Direct Solar"]/30

    df_people = eso_file.to_frame("Zone People Occupant Count")
    df_people.columns = ["People count"]

    df_fan_energy = eso_file.to_frame("Fan Electricity Energy")
    df_fan_energy.columns = ["Supply Fan Energy"]

    df_terminal_temp = eso_file.to_frame("System Node Temperature")

    df_terminal_vol = eso_file.to_frame("System Node Current Density Volume Flow Rate")
    df_terminal_vol.columns = ["Terminal Volume"]

    df_terminal_dens = eso_file.to_frame("System Node Current Density")
    df_terminal_dens.columns = ["Terminal Density"]
    
    df_temp = pd.concat([df_terminal_temp, df_terminal_vol, df_terminal_dens, df_outdoor_temp,df_indoor_temp,df_boiler_out, df_heat_coil, df_solar_dir, df_people,df_fan_energy], axis=1)

    return df_temp

In [4]:
from datetime import datetime, timedelta

def apply_dates(df, start_date, interval):
    """
    Add date to df
    """

    end_date = start_date + timedelta(minutes=interval*len(df)-1)    
    date_range = pd.date_range(start=start_date, end=end_date, freq=f"{interval}T")
    
    df.index = date_range
    
    return df

In [5]:
import os
import glob

def get_latest_dir(config: dict):
    """
    Find latest created eso file
    """
    name_pattern = f"Eplus-env-{config.get('building_file')}-{config.get('weather_file')}-heating-res*"
    
    matching_directories = glob.glob(name_pattern)
    highest_id = -1
    highest_id_directory = None
    
    for directory in matching_directories:
        try:
            # Extract the ID from the directory name
            directory_id = int(directory.split("-")[-1].replace("res",""))
            
            # Update highest_id and highest_id_directory if a higher ID is found
            if directory_id > highest_id:
                highest_id = directory_id
                highest_id_directory = directory
        except ValueError:
            # Handle cases where the directory name doesn't follow the expected pattern
            pass
    
    # Check if any matching directories were found
    return name_pattern.replace("*", str(highest_id))
   

### Main Routine

In [6]:
config, test_setup = main(config_set=config)

#==============================================================================================#
[ENVIRONMENT] (INFO) : Creating Gymnasium environment... [5zone_unitary_heat_only-const-heating]
#==============================================================================================#
[MODELING] (INFO) : Experiment working directory created [/efficientIO/ai/testing/Eplus-env-5zone_unitary_heat_only-const-heating-res289]
[MODELING] (INFO) : runperiod established: {'start_day': 14, 'start_month': 1, 'start_year': 2018, 'end_day': 15, 'end_month': 1, 'end_year': 2018, 'start_weekday': 1, 'n_steps_per_hour': 4}
[MODELING] (INFO) : Episode length (seconds): 172800.0
[MODELING] (INFO) : timestep size (seconds): 900.0
[MODELING] (INFO) : timesteps per episode: 192
[MODELING] (INFO) : Model Config is correct.
[ENVIRONMENT] (INFO) : Environment 5zone_unitary_heat_only-const-heating created successfully.


[I 2023-10-04 13:20:40,430] A new study created in RDB with name: 10/04 13:20:40 5zone_unitary_heat_only-const- Kp:(0.3, 0.6)/0.1 Ki:(0.0004, 0.0012)/0.0004 Kd:(0.0, 25.0)/12.5


#----------------------------------------------------------------------------------------------#
[ENVIRONMENT] (INFO) : Starting a new episode... [5zone_unitary_heat_only-const-heating] [Episode 1]
#----------------------------------------------------------------------------------------------#
[MODELING] (INFO) : Episode directory created [/efficientIO/ai/testing/Eplus-env-5zone_unitary_heat_only-const-heating-res289/Eplus-env-sub_run1]
[MODELING] (INFO) : Weather file AUT_WI_Wien-Innere.Stadt.110340_TMYx.2007-2021_day_const_temp.epw used.
[MODELING] (INFO) : Updated building model with whole Output:Variable available names
[MODELING] (INFO) : Updated building model with whole Output:Meter available names
[MODELING] (INFO) : Extra config: runperiod updated to {'apply_weekend_holiday_rule': 'No', 'begin_day_of_month': 1, 'begin_month': 10, 'day_of_week_for_start_day': 'Tuesday', 'end_day_of_month': 9, 'begin_year': 2018, 'end_year': 2018, 'end_month': 10, 'use_weather_file_daylight_savi

/usr/local/lib/python3.10/dist-packages/opyplus/weather_data/weather_data.py:493: FutureWarning:

the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.



[ENVIRONMENT] (INFO) : Saving episode output path... [/efficientIO/ai/testing/Eplus-env-5zone_unitary_heat_only-const-heating-res289/Eplus-env-sub_run1/output]
[SIMULATOR] (INFO) : Running EnergyPlus with args: ['-w', '/efficientIO/ai/testing/Eplus-env-5zone_unitary_heat_only-const-heating-res289/Eplus-env-sub_run1/AUT_WI_Wien-Innere.Stadt.110340_TMYx.2007-2021_day_const_temp_Random_1.0_0.0_0.001.epw', '-d', '/efficientIO/ai/testing/Eplus-env-5zone_unitary_heat_only-const-heating-res289/Eplus-env-sub_run1/output', '/efficientIO/ai/testing/Eplus-env-5zone_unitary_heat_only-const-heating-res289/Eplus-env-sub_run1/HVAC-5ZoneWaterHeatUnitary.epJSON']
[ENVIRONMENT] (INFO) : Episode 1 started.
[SIMULATOR] (INFO) : handlers initialized.
[SIMULATOR] (INFO) : handlers are ready.
[SIMULATOR] (INFO) : System is ready.
Progress: |***********************************************************************************----------------| 83%
[ENVIRONMENT] (INFO) : Environment closed. [5zone_unitary_heat_on

In [7]:
df = pd.DataFrame(test_setup.setpoint_history, columns=["Datetime","Setpoint"])

# get path to latest eso-file
f_path=os.path.join(f"{get_latest_dir(config)}", f"Eplus-env-sub_run1", "output","eplusout.eso")

# create df from eso file
f = eso.read_from_path(f_path)
df_temps = create_dataframe_from_eso(f)


# calc number of steps
#run_len = config.get("timesteps_per_hour")*config.get("hours_per_trial")*config.get("trials")

# adjust dataframe
#df_temps = df_temps[0:run_len]
df = pd.DataFrame(test_setup.setpoint_history, columns=["Datetime","Setpoint"])
df_temps = df_temps[0:len(df)]

df_final = pd.concat([df, df_temps], axis=1)
       


In [34]:
data = [{"Trial": k, "start_time": v.start_time, "end_time": v.end_time, "score": v.score, "Parameters":
        v.parameters} for k, v in test_setup.optimization_stats[test_setup.study_name].trials.items()]

data = pd.DataFrame(data)
data['Mean Error'] = data["score"]/(test_setup._hours_per_trials*test_setup._timesteps_per_hour)

parameters = pd.DataFrame(data["Parameters"].apply(lambda item: {i.name: i.value for i in item}).tolist())

stats = pd.concat([data, parameters], axis=1)

stats.drop("Parameters", axis=1,inplace=True)
stats.set_index("Trial", inplace=True)

# Find the index with the lowest value in the "Score" column
lowest_index = stats['score'].idxmin()

# Add an asterisk to the index label
stats = stats.rename(index={lowest_index: f'{lowest_index}*'})

stats = stats.rename(columns={
    'start_time': 'Start Time',
    'end_time': 'End Time',
    'score': 'Error',
    'Kp': 'Kp',
    'Ki': 'Ki',
    'Kd': 'Kd'
})

stats

,Start Time,End Time,Error,Mean Error,Kp,Ki,Kd
Trial,,,,,,,
0,2008-10-01 00:00:00,2008-10-01 12:00:00,135.319411,0.187944,0.5,0.0008,25.0
1,2008-10-01 12:00:00,2008-10-02 00:00:00,129.534867,0.179910,0.5,0.0008,12.5
2,2008-10-02 00:00:00,2008-10-02 12:00:00,134.582133,0.186920,0.4,0.0008,12.5
3,2008-10-02 12:00:00,2008-10-03 00:00:00,122.491824,0.170128,0.6,0.0008,12.5
4,2008-10-03 00:00:00,2008-10-03 12:00:00,92.541293,0.128530,0.4,0.0008,12.5
5,2008-10-03 12:00:00,2008-10-04 00:00:00,126.517651,0.175719,0.6,0.0012,0.0
6,2008-10-04 00:00:00,2008-10-04 12:00:00,62.348627,0.086595,0.5,0.0004,12.5
7,2008-10-04 12:00:00,2008-10-05 00:00:00,178.917025,0.248496,0.3,0.0008,12.5
8*,2008-10-05 00:00:00,2008-10-05 12:00:00,21.197954,0.029442,0.5,0.0004,25.0


In [33]:
df_final['Datetime'] = pd.to_datetime(df_final['Datetime'])

# create plot
import plotly.express as px
import plotly.graph_objects as go
from datetime import timedelta
fig = px.line(df_final,x="Datetime", y=config.get("visualize"))
[fig.add_vline(x=i, line_width=1, line_dash="dash", 
        line_color="grey") for i in stats["End Time"]]

[fig.add_trace(
    go.Scatter(
        x=[x-timedelta(hours=config.get("hours_per_trial")/2)],
        y=[100],  # Adjust the y-coordinate to position the text
        text=[f"Trial {i}"],
        mode="text",
        showlegend=False,
        textfont=dict(color='grey' if i!=lowest_index else 'red' )
    )
) for i,x in enumerate(stats["End Time"])]

fig.show()